### Copyright 2024 Jens Liebehenschel, Frankfurt University of Applied Sciences, FB2, Computer Science
### No liability or warranty; only for educational and non-commercial purposes
### See some basic hints for working with Jupyter notebooks in README.md
## A* algorithm for finding a path between two nodes of a graph

### Important remarks before we begin
#### In many educational resources A* works on graphs that are based on grid-like graphs. Here is a generic algorithm that is able to work with "any" kind of graph, also the grid-like graphs. They need to be provided as any other graph as adjacency matrix.
#### There are several potential pitfalls when defining the graphs, e.g. the used node numbers do not fit to the number of rows and columns of the grid-like graphs. This is not tested here, in case of errors, the program will crash (hopefully in any such case).

## Import math module for using infinity and define a constant x for making input of graphs as adjacency matrix more readable

In [1]:
import math
inf = math.inf
x = -inf # no path from ... to ... in adjacency matrix

## Graph and heuristic distance as n x n adjacency matrices

In [2]:
# graph as adjacency matrix
# the algorithm works with node numbers starting at 0, certainly then can be assigned to names like S, T, ...
#     0  1  2  3  4  5  6  7
#     S  T  U  V  W  X  Y  Z
g = [[0, 4, 7, 5, 3, 6, x, x], # node S
     [x, 0, x, 4, x, 3, 2, x], # node T
     [x, x, 0, x, 1, x, x, 2], # node U
     [x, x, 1, 0, 2, x, 1, x], # node V
     [x, x, x, x, 0, x, 1, 4], # node W
     [x, x, x, x, x, 0, 1, x], # node X
     [x, x, x, x, x, x, 0, 2], # node Y
     [3, x, x, x, x, 1, x, 0]  # node Z
     ]

In [3]:
# n x n adjacency matrix
n = len(g)

In [4]:
# optimal path lengths in g, calculated by Floyd-Warshall algorithm, necessary for heuristics
opt = [[0, 4, 6, 5, 3, 6, 4, 6],
       [7, 0, 5, 4, 6, 3, 2, 4],
       [5, 9, 0,10, 1, 3, 2, 2],
       [6,10, 1, 0, 2, 4, 1, 3],
       [6,10,12,11, 0, 4, 1, 3],
       [6,10,12,11, 9, 0, 1, 3],
       [5, 9,11,10, 8, 3, 0, 2],
       [3, 7, 9, 8, 6, 1, 2, 0]
      ]

In [5]:
# easy to generate heuristics for graph g, also as an adjacency matrix
# just make one smaller than the optimal solution for all paths with higeher cost than 1
h = [x[:] for x in opt] # deepcopy of matrix opt
# now modify values
for i in range(n):
    for j in range(n):
        if h[i][j] > 1:
            h[i][j] -= 1

## Supporting function for priority queue

In [6]:
# implementation as list is not efficient, but easy to understand and simple to implement, good enough here
# much better in practice: priority queue using a heap
def extract_min(q):
    min_index = 0
    for i in range(1, len(q)):
        if heur[q[i]] < heur[q[min_index]]:
            min_index = i
    min_index = q.pop(min_index)
    return min_index

## Variables used be A* algorithm

In [7]:
# predecessor of discovered node
pred = []
# cost to starting node s
cost = []
# heuristics to destination node d
heur = []
# set of finished nodes
f = []
# Queue
q = []

## A* algorithm

In [8]:
def initialization(s, d):
    global pred, cost, heur, f, q
    # predecessor of discovered node
    pred = [-1]*n
    # cost to starting node s
    cost = [inf]*n
    cost[s] = 0
    # cost to destination node d
    heur = [inf]*n
    heur[s] = 0
    # set of finished nodes, none at beginning
    f = []
    # queue with all node at beginning
    q = [i for i in range(n)]

In [9]:
def relax(u, v, d):
    if cost[v] > cost[u] + g[u][v]:
        cost[v] = cost[u] + g[u][v]
        heur[v] = cost[v] + h[v][d]
        pred[v] = u

In [10]:
# construct the path from node u to node v (starting at the destination v, moving towards u)
def get_path(u, v):
    if pred[v] == -1:
        return []
    # start from last node
    path = [v]
    while u != v:
        v = pred[v]
        path.insert(0,v)
    return path

In [11]:
def a_star(s, d):
    initialization(s,d)
    while len(q) > 0:
        u = extract_min(q)
        if u == d:
            return cost[d]
        else:
            f.append(u)
            for v in range(n):
                if g[u][v] > 0:
                    relax(u,v,d)

## Test correctness of algorithm be determination of all distances and compare with optimal solution calculated by Floyd-Warshall algorithm

In [12]:
# empty matrix for storing the paths with lowest cost
result = [[-1]*n for _ in range(n)]

for i in range(n):
    for j in range(n):
        result[i][j] = a_star(i,j)
        #print(get_path(i,j))
print ("Results ok? ->", opt == result)

Results ok? -> True


## Generate test data as grid like graphs and test algorithm

## First test: First graph in A-starVisualization.ipynb as of 20.04.2024 (section "Example")

### Generate the graph

In [13]:
# notation
# this is an undirected graph
# 7x7 Matrix with node numbers below, 49 nodes in total
# [ 0 ...  6]
# [ 7 ... 13]
# [14 ... 20]
# [21 ... 27]
# [28 ... 34]
# [35 ... 41]
# [42 ... 48]

In [14]:
# dimensions of grid-based graph
m = 7   # square matrix with m cells per row and column
n = m*m # number of nodes in graph

In [15]:
# supporting functions for better understandability of the code
# m is the number of rows of the graph!
def get_row(node_nr):
    return node_nr // m
def get_col(node_nr):
    return node_nr % m
def get_node_nr(row, col):
    return m * row + col

In [16]:
# step 1: initialize empty graph (no connection between nodes)
g = [[inf]*n for _ in range(n)]

In [17]:
# step 2: add horizontal and vertical connections, the cost is always identical
unit_cost = 1
def add_all_connections():
    for i in range(m):
        for j in range(m):
            # horizontal, except for last column (since there is no "next" column)
            if j < m-1:
                g[get_node_nr(i,j)][get_node_nr(i,j+1)] = unit_cost
                g[get_node_nr(i,j+1)][get_node_nr(i,j)] = unit_cost
            # vertical, except for last row (since there is no "next" row)
            if i < m-1:
                g[get_node_nr(i,j)][get_node_nr(i+1,j)] = unit_cost
                g[get_node_nr(i+1,j)][get_node_nr(i,j)] = unit_cost
add_all_connections()

In [18]:
# nodes -- are not existing in the graph and need to be removed now
# [ 0  1  2  3  4  5  6]
# [ 7  8  9 10 -- 12 13]
# [14 15 16 -- 18 19 20]
# [ s 22 23 -- 25 26 27]
# [28 29 30 -- 32 33  d]
# [35 36 37 38 39 -- 41]
# [-- -- -- -- 46 47 48]

In [19]:
# step 3: remove added connections from and to non-existing nodes
def remove_nodes(nodes_to_be_removed):
    for node in nodes_to_be_removed:
        # remove the connections from and to this node
        # iterate over four neighbor cells
        # set initial inf value, if the neighbor exists in the graph (matrix)
        for i in [-1, 1]: # first up and down (row) ...
            if get_row(node)+i >= 0 and get_row(node)+i < m:
                g[node][get_node_nr(get_row(node)+i,get_col(node))] = inf
                g[get_node_nr(get_row(node)+i,get_col(node))][node] = inf
        for j in [-1, 1]: # ... then left and right (column)
            if get_col(node)+j >= 0 and get_col(node)+j < m:
                g[node][get_node_nr(get_row(node),get_col(node)+j)] = inf
                g[get_node_nr(get_row(node),get_col(node)+j)][node] = inf
remove_nodes([11, 17, 24, 31, 40, 42, 43, 44, 45])

In [20]:
# step 4: 0's on diagonal, i. e. no cost to get to the node itself
def set_diag_to_0():
    for i in range(n):
        g[i][i] = 0
set_diag_to_0()

In [21]:
# initialize heuristics matrix
# instead of the euclidian distance we use the number of nodes in a grid without missing nodes
h = [[inf]*n for _ in range(n)]
for i in range(n):
    for j in range(n):
        h[i][j] = abs(get_row(i)-get_row(j)) + abs(get_col(i)-get_col(j))

In [22]:
# set start s and destination node d
s = 21
d = 34

In [23]:
# run the A* algorithm and output cost and path
print("cost:", a_star(s,d))
path = get_path(s,d)
print("Note: start s and destination d are first and last elements in the lists")
print(path)
print([(get_row(node),get_col(node)) for node in path])

cost: 9
Note: start s and destination d are first and last elements in the lists
[21, 22, 23, 30, 37, 38, 39, 32, 33, 34]
[(3, 0), (3, 1), (3, 2), (4, 2), (5, 2), (5, 3), (5, 4), (4, 4), (4, 5), (4, 6)]


### Your tests here ...

## Second test, based on graph size of first test, only other non-existing cells, reuse the data and perform only necessary changes, m and n are unchanged here!

In [24]:
# nodes -- are not existing in the graph
# [ 0  1  2  3  4  5  6]
# [ 7 -- -- -- -- -- 13]
# [14 15 -- 17 18 19 20]
# [ s 22 -- 24 -- 26 27]
# [28 29 -- 31 32 -- 34]
# [35 36 37 --  d -- 41]
# [42 43 44 45 -- 47 48]

In [25]:
# reinitialize g
# reuse g
# only steps 2 to 4 necessary
add_all_connections()
remove_nodes([8, 9, 10, 11, 12, 16, 23, 25, 30, 33, 38, 40, 46])
set_diag_to_0()

In [26]:
# set start s and destination node d
s = 21
d = 39

In [27]:
# run the A* algorithm and output cost and path
print("cost:", a_star(s,d))
path = get_path(s,d)
print("Note: start s and destination d are first and last elements in the lists")
print(path)
print([(get_row(node),get_col(node)) for node in path])

cost: 18
Note: start s and destination d are first and last elements in the lists
[21, 14, 7, 0, 1, 2, 3, 4, 5, 6, 13, 20, 19, 18, 17, 24, 31, 32, 39]
[(3, 0), (2, 0), (1, 0), (0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (1, 6), (2, 6), (2, 5), (2, 4), (2, 3), (3, 3), (4, 3), (4, 4), (5, 4)]


### ... and here ...